In [52]:
#transformation du texte sous forme de liste en string
def jonction(liste):
  result = ""
  for token in liste:
    result = result + token + " "
  return result

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from joblib import load,dump
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
import time
import os
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

def optimisation_model(df_post_traduction,colonne_a_vectoriser,max_features,k_best,joblib_path_suivi_metrique,grid_search_cv):
    #import du df
    df = load(df_post_traduction)
    #transformation de la future colonne feature en chaine de caractère
    df[colonne_a_vectoriser]=df[colonne_a_vectoriser].apply(jonction)
    #séparation des features et de la cible
    data = df[colonne_a_vectoriser]
    target = df["prdtypecode"]
    #séparation de train et test
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42, stratify = target)
    #vectorisation selon un nombre max de mots les plus fréquents
    vectorizer = CountVectorizer(max_features=max_features)
    X_train = vectorizer.fit_transform(X_train).todense()
    X_test = vectorizer.transform(X_test).todense()
    scaler = StandardScaler().fit(np.asarray(X_train))
    X_train = scaler.transform(np.asarray(X_train))
    X_test = scaler.transform(np.asarray(X_test))
    sel = SelectKBest(k=k_best)
    sel.fit(np.asarray(X_train),y_train)
    X_train = sel.transform(np.asarray(X_train))
    X_test = sel.transform(np.asarray(X_test))
    if os.path.exists(joblib_path_suivi_metrique):
        df_import = load(joblib_path_suivi_metrique)
        print("récupération du df existant")
    else:
        df_import = pd.DataFrame(columns=["Max_features","K_best","Grid","Hyperparamètres","Best_param", "Accuracy", "F1_weighted", "F1_macro", "Duree en sec"])
        print("création d'un dataframe")
    score = []
    print("debut du grid:",grid_search_cv)
    debut = time.time()
    grid_search_cv.fit(np.asarray(X_train),y_train)
    print("fin du grid:",grid_search_cv)
    print("debut meilleur param:",grid_search_cv.best_params_)
    best_params = grid_search_cv.best_params_
    best_rf_model = RandomForestClassifier(**best_params)
    best_rf_model.fit(np.asarray(X_train), y_train)
    y_pred  = best_rf_model.predict(np.asarray(X_test))
    print("fin meilleur param")
    accuracy = accuracy_score(y_test, y_pred)
    f1_weighted = f1_score(y_test, y_pred, average='weighted')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    fin = time.time()
    duree = fin - debut
    model_scores = { 
            "Max_features": f"{max_features}",
            "K_best":f"{k_best}",
            "Grid": f"{grid_search_cv}",
            "Hyperparamètres":f"{param_grid}",
            "Best_param":f"{best_params}",
            "Accuracy": accuracy,
            "F1_weighted": f1_weighted,
            "F1_macro": f1_macro,
            "Duree en sec": duree}
    score.append(model_scores)
    df_score = pd.DataFrame(score)
    df = pd.concat([df_import, df_score], ignore_index=True)
    return df

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
rdf = RandomForestClassifier(n_jobs = -1,random_state = 23)

param_grid = {
    'n_estimators': [400],  # Nombre d'arbres dans la forêt
    'criterion': ['gini'],  # Métrique de qualité de la division
    'max_depth': [None],  # Profondeur maximale de chaque arbre
    'min_samples_split': [5],  # Nombre minimal d'échantillons requis pour diviser un nœud
    'min_samples_leaf': [1],  # Nombre minimal d'échantillons requis pour être une feuille
    'max_features': ['log2'],  # Nombre maximal de fonctionnalités à considérer pour la division
    'bootstrap': [False],  # Si l'échantillonnage bootstrap doit être utilisé
    'class_weight': ['balanced'],  # Poids des classes pour la correction de déséquilibre
    'random_state': [23],  # Seed pour la reproductibilité
    'n_jobs': [-1]  # Utilisation de tous les cœurs de CPU}
}

gcv = GridSearchCV(estimator=rdf, param_grid = param_grid,cv=5, n_jobs=-1, scoring="f1_weighted")




In [ ]:
#param_grid = {
    'n_estimators': [100, 200, 300],  # Nombre d'arbres 
    'criterion': ['gini', 'entropy',],  # Métrique de qualité de la division
    'max_depth': [None, 10, 20, 30],  # Profondeur maximale de chaque arbre
    'min_samples_split': [2, 5, 10],  # Nombre minimal d'échantillons requis pour diviser un nœud
    'min_samples_leaf': [1, 2, 4],  # Nombre minimal d'échantillons requis pour être une feuille
    'max_features': ['auto', 'sqrt', 'log2'],  # Nombre maximal de fonctionnalités à considérer pour la division
    'bootstrap': [True, False],  # Si l'échantillonnage bootstrap doit être utilisé
    'class_weight': [None, 'balanced', 'balanced_subsample'],  # Poids des classes pour la correction de déséquilibre
    'random_state': [23],  # Seed pour la reproductibilité
    'n_jobs': [-1]  # Utilisation de tous les cœurs de CPU}

In [32]:
df_post_traduction = r"C:\Users\franc\AutoML\df_train_post_trad.joblib"
colonne_a_vectoriser = "mots_stem_sans_chiffres"
max_features = 4000
k_best = 3000
joblib_path_suivi_metrique = r"C:\Users\franc\AutoML\df_optimisation.joblib"
grid_search_cv = gcv
df_optimisation = optimisation_model(df_post_traduction,colonne_a_vectoriser,max_features,k_best,joblib_path_suivi_metrique,grid_search_cv)
dump(df_optimisation,r"C:\Users\franc\AutoML\df_optimisation.joblib")

récupération du df existant
debut du grid: GridSearchCV(cv=5, estimator=RandomForestClassifier(n_jobs=-1, random_state=23),
             n_jobs=-1,
             param_grid={'bootstrap': [False], 'class_weight': ['balanced'],
                         'criterion': ['gini'], 'max_depth': [None],
                         'max_features': ['log2'], 'min_samples_leaf': [1],
                         'min_samples_split': [5], 'n_estimators': [400],
                         'n_jobs': [-1], 'random_state': [23]},
             scoring='f1_weighted')
fin du grid: GridSearchCV(cv=5, estimator=RandomForestClassifier(n_jobs=-1, random_state=23),
             n_jobs=-1,
             param_grid={'bootstrap': [False], 'class_weight': ['balanced'],
                         'criterion': ['gini'], 'max_depth': [None],
                         'max_features': ['log2'], 'min_samples_leaf': [1],
                         'min_samples_split': [5], 'n_estimators': [400],
                         'n_jobs': [-1], '

['C:\\Users\\franc\\AutoML\\df_optimisation.joblib']

In [33]:
joblib_path_suivi_metrique = r"C:\Users\franc\AutoML\df_optimisation.joblib"
df_optimisation = load(joblib_path_suivi_metrique)
df_optimisation

,Max_features,K_best,Grid,Hyperparamètres,Best_param,Accuracy,F1_weighted,F1_macro,Duree en sec
0,4000,3000,"GridSearchCV(cv=5, estimator=RandomForestClass...","{'class_weight': ['balanced'], 'random_state':...","{'class_weight': 'balanced', 'criterion': 'gin...",0.772136,0.772812,0.750719,1072.721818
1,4000,3000,"GridSearchCV(cv=5, estimator=RandomForestClass...","{'class_weight': ['balanced'], 'random_state':...","{'class_weight': 'balanced', 'criterion': 'gin...",0.772688,0.773371,0.750832,1872.766705
2,4000,3000,"GridSearchCV(cv=5, estimator=RandomForestClass...","{'class_weight': ['balanced'], 'random_state':...","{'class_weight': 'balanced', 'criterion': 'gin...",0.772136,0.772812,0.750719,1176.132017
3,4000,3000,"GridSearchCV(cv=5, estimator=RandomForestClass...","{'class_weight': ['balanced'], 'random_state':...","{'class_weight': 'balanced', 'criterion': 'gin...",0.779381,0.780637,0.757437,1647.827343
4,4000,3000,"GridSearchCV(cv=5, estimator=RandomForestClass...","{'class_weight': ['balanced'], 'random_state':...","{'class_weight': 'balanced', 'criterion': 'gin...",0.779381,0.780637,0.757437,1703.849127
5,4000,3000,"GridSearchCV(cv=5, estimator=RandomForestClass...","{'class_weight': ['balanced'], 'random_state':...","{'class_weight': 'balanced', 'criterion': 'gin...",0.782758,0.782434,0.764278,773.802725
6,4000,3000,"GridSearchCV(cv=5, estimator=RandomForestClass...","{'class_weight': ['balanced'], 'random_state':...","{'bootstrap': False, 'class_weight': 'balanced...",0.782881,0.782675,0.764457,714.752244
7,4000,3000,"GridSearchCV(cv=5, estimator=RandomForestClass...","{'class_weight': [None, 'balanced', 'balanced_...","{'bootstrap': False, 'class_weight': 'balanced...",0.782881,0.782675,0.764457,1777.478430
8,4000,3000,"GridSearchCV(cv=5, estimator=RandomForestClass...","{'n_estimators': [100, 200, 300], 'criterion':...","{'criterion': 'gini', 'n_estimators': 300}",0.772013,0.771371,0.750619,5582.291008
9,4000,3000,"GridSearchCV(cv=5, estimator=RandomForestClass...","{'n_estimators': [50, 100, 200, 300], 'criteri...","{'bootstrap': False, 'class_weight': 'balanced...",0.775636,0.775725,0.757550,2151.491883


## Regression logistique

In [40]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
param_grid = {
    'penalty': ["l2",None], 
    'C': [0.001, 0.01, 0.1, 1, 10, 100], 
    'solver': ["lbfgs"],
    'max_iter': [100],
    'class_weight':['balanced'],
    'random_state': [23]}
gcv = GridSearchCV(estimator=lr, param_grid = param_grid,cv=5, n_jobs=-1, scoring="f1_weighted")  
"""{'C': 0.01, 'class_weight': 'balanced', 'max_iter': 100, 'penalty': 'l2', 'random_state': 23, 'solver': 'lbfgs'}"""

In [ ]:
"""param_grid = {
    'penalty': ['l1', 'l2'], # Type de régularisation : l1 (Lasso) ou l2 (Ridge)
    'C': [0.001, 0.01, 0.1, 1, 10, 100], # Inverse de la force de régularisation (plus C est grand, moins la régularisation est forte)
    'solver': ['liblinear', 'saga'], # Algorithme d'optimisation (liblinear pour les petites données, saga pour les grandes données)
    'max_iter': [100, 200, 300] # Nombre maximal d'itérations pour la convergence de l'optimisation}

In [41]:
df_post_traduction = r"C:\Users\franc\AutoML\df_train_post_trad.joblib"
colonne_a_vectoriser = "mots_stem_sans_chiffres"
max_features = 4000
k_best = 3000
joblib_path_suivi_metrique = r"C:\Users\franc\AutoML\df_optimisation.joblib"
grid_search_cv = gcv
df_optimisation = optimisation_model(df_post_traduction,colonne_a_vectoriser,max_features,k_best,joblib_path_suivi_metrique,grid_search_cv)
dump(df_optimisation,r"C:\Users\franc\AutoML\df_optimisation.joblib")

récupération du df existant
debut du grid: GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'class_weight': ['balanced'], 'max_iter': [100],
                         'penalty': ['l2', None], 'random_state': [23],
                         'solver': ['lbfgs']},
             scoring='f1_weighted')
fin du grid: GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'class_weight': ['balanced'], 'max_iter': [100],
                         'penalty': ['l2', None], 'random_state': [23],
                         'solver': ['lbfgs']},
             scoring='f1_weighted')
debut meilleur param: {'C': 0.01, 'class_weight': 'balanced', 'max_iter': 100, 'penalty': 'l2', 'random_state': 23, 'solver': 'lbfgs'}


C:\Users\franc\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


TypeError: __init__() got an unexpected keyword argument 'C'

### Sans grid search pour tester quelques combinaison 

In [42]:
#transformation du texte sous forme de liste en string
def jonction(liste):
  result = ""
  for token in liste:
    result = result + token + " "
  return result

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from joblib import load,dump
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
import time
import os
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

def test_model(df_post_traduction,colonne_a_vectoriser,max_features,k_best,joblib_path_suivi_metrique,liste_modele):
    #import du df
    df = load(df_post_traduction)
    #transformation de la future colonne feature en chaine de caractère
    df[colonne_a_vectoriser]=df[colonne_a_vectoriser].apply(jonction)
    #séparation des features et de la cible
    data = df[colonne_a_vectoriser]
    target = df["prdtypecode"]
    #séparation de train et test
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42, stratify = target)
    #vectorisation selon un nombre max de mots les plus fréquents
    vectorizer = CountVectorizer(max_features=max_features)
    X_train = vectorizer.fit_transform(X_train).todense()
    X_test = vectorizer.transform(X_test).todense()
    scaler = StandardScaler().fit(np.asarray(X_train))
    X_train = scaler.transform(np.asarray(X_train))
    X_test = scaler.transform(np.asarray(X_test))
    sel = SelectKBest(k=k_best)
    sel.fit(np.asarray(X_train),y_train)
    X_train = sel.transform(np.asarray(X_train))
    X_test = sel.transform(np.asarray(X_test))
    if os.path.exists(joblib_path_suivi_metrique):
        df_import = load(joblib_path_suivi_metrique)
        print("récupération du df existant")
    else:
        df_import = pd.DataFrame(columns=["Colonne vectorisée","Max_features","K_best","Model", "Accuracy", "F1_weighted", "F1_macro", "Duree en sec"])
        print("création d'un dataframe")
    score = []
    for model in liste_modele:
        print("debut du modèle:",model)
        debut = time.time()
        model.fit(np.asarray(X_train),y_train)
        y_pred = model.predict(np.asarray(X_test))
        print("fin du modèle:",model)
        accuracy = accuracy_score(y_test, y_pred)
        f1_weighted = f1_score(y_test, y_pred, average='weighted')
        f1_macro = f1_score(y_test, y_pred, average='macro')
        fin = time.time()
        duree = fin - debut
        model_scores = {
            "Colonne vectorisée":f"{colonne_a_vectoriser}",
            "Max_features": f"{max_features}",
            "K_best":f"{k_best}",
            "Model": f"{model}",
            "Accuracy": accuracy,
            "F1_weighted": f1_weighted,
            "F1_macro": f1_macro,
            "Duree en sec": duree}
        score.append(model_scores)
    df_score = pd.DataFrame(score)
    df = pd.concat([df_import, df_score], ignore_index=True)
    return df


In [51]:
df_post_traduction = r"C:\Users\franc\AutoML\df_train_post_trad.joblib"
max_features = 4000
k_best = 3000
joblib_path_suivi_metrique = r"C:\Users\franc\AutoML\df_score.joblib"
colonne_a_vectoriser = "mots_stem_sans_chiffres"
lr3 = LogisticRegression(multi_class="multinomial",C= 0.01, class_weight= 'balanced', max_iter= 500, penalty= 'l2', random_state= 23, solver= 'saga')
liste_modele = [lr3]
df_score = test_model(df_post_traduction,colonne_a_vectoriser,max_features,k_best,joblib_path_suivi_metrique,liste_modele)
dump(df_score,r"C:\Users\franc\AutoML\df_score.joblib")

récupération du df existant
debut du modèle: LogisticRegression(C=0.01, class_weight='balanced', max_iter=500,
                   multi_class='multinomial', random_state=23, solver='saga')


C:\Users\franc\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


fin du modèle: LogisticRegression(C=0.01, class_weight='balanced', max_iter=500,
                   multi_class='multinomial', random_state=23, solver='saga')


['C:\\Users\\franc\\AutoML\\df_score.joblib']

### BalancedRandomForestClassifier avec les paramètres du meilleure RandomForestClassifier 

In [54]:
from imblearn.ensemble import BalancedRandomForestClassifier
df_post_traduction = r"C:\Users\franc\AutoML\df_train_post_trad.joblib"
max_features = 4000
k_best = 3000
joblib_path_suivi_metrique = r"C:\Users\franc\AutoML\df_score.joblib"
colonne_a_vectoriser = "mots_stem_sans_chiffres"
brfc = BalancedRandomForestClassifier(n_estimators=1000,  criterion='gini',  max_depth= None,  
                               min_samples_split= 5, min_samples_leaf = 1, max_features='log2',  
                               bootstrap=False,  class_weight='balanced',  random_state=23,  n_jobs=-1)
liste_modele = [brfc]
df_score = test_model(df_post_traduction,colonne_a_vectoriser,max_features,k_best,joblib_path_suivi_metrique,liste_modele)
dump(df_score,r"C:\Users\franc\AutoML\df_score.joblib")

récupération du df existant
debut du modèle: BalancedRandomForestClassifier(bootstrap=False, class_weight='balanced',
                               max_features='log2', min_samples_split=5,
                               n_estimators=1000, n_jobs=-1, random_state=23)


C:\Users\franc\anaconda3\lib\site-packages\imblearn\ensemble\_forest.py:546: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
C:\Users\franc\anaconda3\lib\site-packages\imblearn\ensemble\_forest.py:558: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(


fin du modèle: BalancedRandomForestClassifier(bootstrap=False, class_weight='balanced',
                               max_features='log2', min_samples_split=5,
                               n_estimators=1000, n_jobs=-1, random_state=23)


['C:\\Users\\franc\\AutoML\\df_score.joblib']

In [55]:
df = load("df_score.joblib")
df

,Colonne vectorisée,Max_features,K_best,Model,Accuracy,F1_weighted,F1_macro,Duree en sec
0,mots_stem_sans_chiffres,4000,3000,"LogisticRegression(class_weight='balanced', ra...",0.737689,0.742634,0.713987,33.572307
1,mots_stem_sans_chiffres,4000,3000,KNeighborsClassifier(n_jobs=-1),0.573560,0.580204,0.539838,46.121506
2,mots_stem_sans_chiffres,4000,3000,DecisionTreeClassifier(class_weight='balanced'...,0.699067,0.702138,0.673113,86.763731
3,mots_stem_sans_chiffres,4000,3000,RandomForestClassifier(class_weight='balanced'...,0.772136,0.772812,0.750719,80.730617
4,mots_stem_sans_chiffres,4000,3000,BalancedRandomForestClassifier(class_weight='b...,0.709382,0.718596,0.679694,150.788608
5,mots_stem,4000,3000,"LogisticRegression(class_weight='balanced', ra...",0.737996,0.742530,0.712957,34.400597
6,mots_stem,4000,3000,KNeighborsClassifier(n_jobs=-1),0.562937,0.573612,0.534399,46.437106
7,mots_stem,4000,3000,DecisionTreeClassifier(class_weight='balanced'...,0.699435,0.702849,0.672202,84.432850
8,mots_stem,4000,3000,RandomForestClassifier(class_weight='balanced'...,0.770109,0.771216,0.748568,80.628054
9,mots_stem,4000,3000,BalancedRandomForestClassifier(class_weight='b...,0.708523,0.716657,0.682357,143.169274
